# Demo for SPEL


In [1]:
%tb
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Built-in modules
import re        # Regular expressions
import importlib # MUST BE USED TO RELOAD MODULES
import sys 

# Importing modules for SPEL functions 
import utilityFunctions as uf
import edit_files as ef 
import fortran_modules as fm 
from analyze_subroutines import Subroutine

# mod_config : system configuration and static variables. 
# Where E3SM is stored, where unit tests are stored, etc.
from mod_config import default_mods, unittests_dir, scripts_dir, spel_mods_dir
from mod_config import ELM_SRC, spel_output_dir, _bc

No traceback available to show.


SPEL needs a "casename" (what your unit-test is called) and a list of subroutines to create a unit-test for "sub_name_list" (examples are "LakeTemperature", "SoilTemperature", etc...)


In [2]:
# Define Unit Test parameters 
import os 
casename = "notebook_test"  # Name of the test case
case_dir = unittests_dir + casename # Directory to store the test case

# create directory for unit-test if it does not exist
if(not os.path.isdir(f"{case_dir}") ):
    print(f"Making case directory {case_dir}")
    if(not os.path.isdir(f"{unittests_dir}")):
        os.system(f"mkdir {unittests_dir}")
    os.system(f"mkdir {case_dir}")
    preprocess = True

# List of subroutines to be analyzed
# sub_name_list = ["SoilLittVertTransp","LakeTemperature","SoilTemperature"] 
sub_name_list = ["SoilTemperature"] 
sub_name_list = [sub.lower() for sub in sub_name_list]

# variables needed for Unit Test
main_sub_dict = {}  # Dictionary to store all Subroutines in files needed for Unit Test
mod_dict = {} 
subroutines = {k.lower():[] for k in sub_name_list} # Dictionary for User Specified Subroutines

Making case directory /home/mrgex/SPEL_Openacc/scripts/../unit-tests/notebook_test


0

1st step for SPEL is determine which modules are needed for LakeTemperature and edit out the I/O, MPI, and other unneccessary modules. This must be somewhat tailored to ELM.

Currently if I module is not present in the "components/elm/src/" (ELM_SRC) nor in the "share/utils" directories (SHR_SRC), then the module and any dependency on it is automatically removed.

In [3]:
 # List to hold all the modules needed for the unit test
needed_mods = [] 
for s in sub_name_list:
    # Get general info of the subroutine
    subroutines[s] = Subroutine(s,calltree=['elm_drv'])

    # Process by removing certain modules and syntax
    # so that a standalone unit test can be compiled.
    fn = subroutines[s].filepath
    mod_dict, file_list, main_sub_dict = ef.process_for_unit_test(
                fname=fn,
                case_dir=case_dir,
                mod_dict=mod_dict,
                mods=needed_mods,
                required_mods=default_mods,
                main_sub_dict=main_sub_dict,
                overwrite=True,
                verbose=False,
            )

    subroutines[s] = main_sub_dict[s]

Adding clumps to clump_type instances
process_for_unit_test::new mods found ['SoilLittVertTranspMod.F90', 'shr_kind_mod.F90', 'elm_varctl.F90', 'elm_varcon.F90', 'decompMod.F90', 'CNDecompCascadeConType.F90', 'CanopyStateType.F90', 'CNStateType.F90', 'ColumnDataType.F90', 'timeinfoMod.F90', 'elm_varpar.F90', 'TridiagonalMod.F90', 'shr_const_mod.F90', 'domainMod.F90', 'landunit_varcon.F90', 'LandunitType.F90', 'ColumnType.F90', 'VegetationType.F90', 'topounit_varcon.F90', 'GridcellType.F90', 'CH4varcon.F90', 'pftvarcon.F90', 'soilorder_varcon.F90', 'column_varcon.F90', 'UrbanParamsType.F90', 'elm_varsur.F90']
Adding filter to clumpfilter instances
Adding filter_inactive_and_active to clumpfilter instances
process_for_unit_test::new mods found ['CH4Mod.F90', 'EnergyFluxType.F90', 'LakeStateType.F90', 'SoilStateType.F90', 'SolarAbsorbedType.F90', 'VegetationDataType.F90', 'LakeCon.F90', 'QSatMod.F90', 'LakeTemperatureMod.F90', 'SharedParamsMod.F90', 'atm2lndType.F90', 'lnd2atmType.F90', '

In [4]:
# Create dictionary containing all identified user defined types
# 'type-name' : 'DerivedType Object'
type_dict = {}
for modname, mod in mod_dict.items():
    for utype, dtype in mod.defined_types.items():
        type_dict[utype] = dtype

SPEL can print out a module tree showing how the modules are linked for the unit-test subroutines
- The full tree can be pretty difficult to parse, having a cutoff depth is recommended.
- Every module uses 'shr_kind_mod' so that could be suppressed as well.

In [5]:
modtree = fm.print_spel_module_dependencies(mod_dict=mod_dict,subs=subroutines)

# Print the module tree
arrow = "-->"
cutoff_depth = 10 # Only print modules up to this depth
suppress_mod_list = ['shr_kind_mod']
for m in modtree:
    depth = m['depth']
    modname = m['module']
    if(modname in suppress_mod_list):
        continue
    if(depth == 1):
        print(_bc.HEADER + arrow*depth + modname + _bc.ENDC)
    elif(depth <= cutoff_depth):
        print( arrow*depth + modname)

soillittverttranspmod
-->elm_varctl
-->elm_varcon
-->-->shr_const_mod
-->-->elm_varpar
-->-->-->elm_varctl
-->-->elm_varctl
-->decompmod
-->-->elm_varctl
-->-->elm_varcon
-->-->-->shr_const_mod
-->-->-->elm_varpar
-->-->-->-->elm_varctl
-->-->-->elm_varctl
-->-->domainmod
-->-->-->elm_varctl
-->cndecompcascadecontype
-->-->decompmod
-->-->-->elm_varctl
-->-->-->elm_varcon
-->-->-->-->shr_const_mod
-->-->-->-->elm_varpar
-->-->-->-->-->elm_varctl
-->-->-->-->elm_varctl
-->-->-->domainmod
-->-->-->-->elm_varctl
-->-->elm_varpar
-->-->-->elm_varctl
-->canopystatetype
-->-->decompmod
-->-->-->elm_varctl
-->-->-->elm_varcon
-->-->-->-->shr_const_mod
-->-->-->-->elm_varpar
-->-->-->-->-->elm_varctl
-->-->-->-->elm_varctl
-->-->-->domainmod
-->-->-->-->elm_varctl
-->-->landunit_varcon
-->-->elm_varcon
-->-->-->shr_const_mod
-->-->-->elm_varpar
-->-->-->-->elm_varctl
-->-->-->elm_varctl
-->-->elm_varpar
-->-->-->elm_varctl
-->-->elm_varctl
-->-->landunittype
-->-->-->elm_varcon
-->-->-->-->shr

In [7]:
test_mod = mod_dict['soiltemperaturemod']
test_mod.print_module_info()

Module Name: soiltemperaturemod biogeophys/SoilTemperatureMod.F90
Module Depedencies:
use shr_kind_mod-> shr_kind_r8,
use decompmod-> bounds_type,
use elm_varctl-> iulog, use_petsc_thermal_model,
use elm_varcon-> spval, cnfac, cpice, cpliq, denh2o, denice, tfrz, tkwat, tkice, tkair, thk_bedrock, hfus, grav, sb, hvap, capr,
use urbanparamstype-> urbanparams_type,
use atm2lndtype-> atm2lnd_type,
use canopystatetype-> canopystate_type,
use solarabsorbedtype-> solarabs_type,
use soilstatetype-> soilstate_type,
use energyfluxtype-> energyflux_type,
use topounitdatatype-> top_af,
use landunittype-> lun_pp,
use landunitdatatype-> lun_es, lun_ef,
use columntype-> col_pp,
use columndatatype-> col_es, col_ef, col_ws, col_wf,
use vegetationtype-> veg_pp,
use vegetationdatatype-> veg_ef, veg_wf,
use timeinfomod-> all
use perfmod_gpu-> all
use elm_varpar-> nlevsno, nlevgrnd, nlevurb, nlevsoi, max_patch_per_col,
use landunit_varcon-> istice, istice_mec, istsoil, istcrop, istwet,
use column_varcon-> 

In [8]:
for s in sub_name_list:
    # Parsing means getting info on the variables read and written
    # to by the subroutine and any of its callees
    subroutines[s].parse_subroutine(dtype_dict=type_dict,
                                    main_sub_dict=main_sub_dict,verbose=True)

Resolving interface for tridiagonal
 with args: ['bounds', '0', 'nlevdecomp+1', 'jtop', 'num_soilc', 'filter_soilc', 'a_tri', 'b_tri', 'c_tri', 'r_tri', 'conc_trcr']
/home/mrgex/SPEL_Openacc/scripts/../../repo/E3SM/components/elm/src/biogeophys/TridiagonalMod.F90 14   interface Tridiagonal
Couldn't match 0 to any known variable -- assuming xx type
Couldn't match nlevdecomp+1 to any known variable -- assuming xx type
tridiagonal_sr:: 1 Optional arguments found
bounds matches bounds
0 matches lbj
nlevdecomp+1 matches ubj
jtop matches jtop
num_soilc matches numf
filter_soilc matches filter
a_tri matches a
b_tri matches b
c_tri matches c
r_tri matches r
conc_trcr matches u
resolve_interface::Subroutine is tridiagonal_sr
_preprocess_file::New child sub name is: tridiagonal_sr
_preprocess_file::Finished analyzing for soillittverttransp
Resolving interface for tridiagonal
 with args: ['bounds', '-nlevsno + 1', 'nlevlak + nlevgrnd', 'jtop', 'num_lakec', 'filter_lakec', 'a', 'b', 'c1', 'r', 'tx

In [9]:
read_types  = [] 
write_types = []
func_name = "Notebook"
for s in sub_name_list:
    subroutines[s].child_subroutines_analysis(dtype_dict=type_dict,
                                        main_sub_dict=main_sub_dict,verbose=True)

    print(_bc.OKGREEN + f"Derived Type Analysis for {subroutines[s].name}")
    print(f"{func_name}::Read-Only")
    for key in subroutines[s].elmtype_r.keys():
        print(key, subroutines[s].elmtype_r[key])
    print(f"{func_name}::Write-Only")
    for key in subroutines[s].elmtype_w.keys():
        print(key, subroutines[s].elmtype_w[key])
    print(f"{func_name}::Read-Write")
    for key in subroutines[s].elmtype_rw.keys():
        print(key, subroutines[s].elmtype_rw[key])
    print(_bc.ENDC)

    for key in list(subroutines[s].elmtype_r.keys()):
        c13c14 = bool("c13" in key or "c14" in key)
        if c13c14:
            del subroutines[s].elmtype_r[key]
            continue
        if "_inst" in key:
            print(f"error: {key} has _inst")
            sys.exit(1)
        read_types.append(key)

    for key in list(subroutines[s].elmtype_w.keys()):
        c13c14 = bool("c13" in key or "c14" in key)
        if c13c14:
            del subroutines[s].elmtype_w[key]
            continue
        if "_inst" in key:
            print(f"error: {key} has _inst")
            sys.exit(1)
        write_types.append(key)

    for key in list(subroutines[s].elmtype_rw.keys()):
        c13c14 = bool("c13" in key or "c14" in key)
        if c13c14:
            del subroutines[s].elmtype_rw[key]
            continue
        write_types.append(key)

Analyzing child subroutine: tridiagonal_sr
_preprocess_file::Finished analyzing for tridiagonal_sr
Derived Type Analysis for soillittverttransp
Notebook::Read-Only
soillittverttranspparamsinst%som_diffus r
soillittverttranspparamsinst%cryoturb_diffusion_k r
soillittverttranspparamsinst%max_altdepth_cryoturbation r
canopystate_vars%altmax_col r
canopystate_vars%altmax_lastyear_col r
decomp_cascade_con%spinup_factor r
decomp_cascade_con%is_cwd r
cnstate_vars%scalaravg_col r
col_cf%decomp_cpools_sourcesink r
col_nf%decomp_npools_sourcesink r
col_pf%decomp_ppools_sourcesink r
c13_col_cf%decomp_cpools_sourcesink r
c14_col_cf%decomp_cpools_sourcesink r
Notebook::Write-Only
Notebook::Read-Write
cnstate_vars%som_diffus_coef_col rw
cnstate_vars%som_adv_coef_col rw
col_cs%decomp_cpools_vr rw
col_ns%decomp_npools_vr rw
col_ps%decomp_ppools_vr rw
c13_col_cs%decomp_cpools_vr rw
c14_col_cs%decomp_cpools_vr rw
col_cf%decomp_cpools_transport_tendency rw
col_nf%decomp_npools_transport_tendency rw
col_p

In [10]:
# Make sure physical properties types are read/written:
list_pp = ['veg_pp','lun_pp','col_pp','grc_pp','top_pp']

print("read_types:",read_types)
print("write_types:",write_types)

aggregated_elmtypes_list = []
for x in read_types:
    dtype_inst = x.split('%')[0]
    if(dtype_inst not in aggregated_elmtypes_list):
        aggregated_elmtypes_list.append(dtype_inst)    
for x in write_types:
    dtype_inst = x.split('%')[0]
    if(dtype_inst not in aggregated_elmtypes_list):
        aggregated_elmtypes_list.append(dtype_inst)

# for l in list_pp:
#     aggregated_elmtypes_list.append(l)
print("list of global vars:",aggregated_elmtypes_list)

read_types: ['soillittverttranspparamsinst%som_diffus', 'soillittverttranspparamsinst%cryoturb_diffusion_k', 'soillittverttranspparamsinst%max_altdepth_cryoturbation', 'canopystate_vars%altmax_col', 'canopystate_vars%altmax_lastyear_col', 'decomp_cascade_con%spinup_factor', 'decomp_cascade_con%is_cwd', 'cnstate_vars%scalaravg_col', 'col_cf%decomp_cpools_sourcesink', 'col_nf%decomp_npools_sourcesink', 'col_pf%decomp_ppools_sourcesink', 'bounds%endc', 'bounds%begc', 'col_pp%snl', 'veg_pp%column', 'solarabs_vars%fsds_nir_d_patch', 'solarabs_vars%fsds_nir_i_patch', 'solarabs_vars%fsr_nir_d_patch', 'solarabs_vars%fsr_nir_i_patch', 'solarabs_vars%sabg_patch', 'col_pp%z_lake', 'lakestate_vars%ws_col', 'lakestate_vars%ks_col', 'col_es%t_grnd', 'col_pp%lakedepth', 'lakestate_vars%etal_col', 'col_pp%dz_lake', 'solarabs_vars%sabg_lyr_patch', 'col_pp%z', 'col_pp%dz', 'lakestate_vars%lake_raw_col', 'soilstate_vars%tksatu_col', 'soilstate_vars%tkmg_col', 'soilstate_vars%watsat_col', 'soilstate_vars%

In [11]:
from UnitTestforELM import set_active_variables
instance_to_user_type = {}
elm_inst_vars = {}
for type_name, dtype in type_dict.items():
    if('bounds' in type_name): 
        continue
    if(not dtype.instances):
        print(f"Warning: no instances found for {type_name}")
        cmd = f'grep -rin -E "^[[:space:]]*(type)[[:space:]]*\({type_name}" {ELM_SRC}/main/elm_instMod.F90'
        output = sp.getoutput(cmd)
        print(f"output: {output}")
        if(output):
            output = output.split('\n')
            if(len(output) > 1):
                print(f"Warning: multiple instances found for {type_name}")
                print(output)
                sys.exit(1)
            line = output[0]
            line = line.replace('::','')
            line = line.split(':')
            
            decl = line[1].strip()
            decl = decl.split()
            var = decl[1]
            new_inst = Variable(type_name,var,subgrid='?',ln=0,dim=0,declaration='elm_instMod')
            dtype.instances.append(new_inst)
            elm_inst_vars[var] = dtype
        else:
            print(f"Warning: no instances found for {type_name}")
    for instance in dtype.instances:
        instance_to_user_type[instance.name] = type_name

dtype_info_list = []
    
for s in sub_name_list:
    set_active_variables(type_dict,instance_to_user_type,
                            subroutines[s].elmtype_r,dtype_info_list)
    set_active_variables(type_dict,instance_to_user_type,
                            subroutines[s].elmtype_w,dtype_info_list)
    set_active_variables(type_dict,instance_to_user_type,
                            subroutines[s].elmtype_rw,dtype_info_list)
    
print(dtype_info_list)

[['soillittverttranspparamsinst', 'som_diffus', 'real', '0D'], ['soillittverttranspparamsinst', 'cryoturb_diffusion_k', 'real', '0D'], ['soillittverttranspparamsinst', 'max_altdepth_cryoturbation', 'real', '0D'], ['canopystate_vars', 'altmax_col', 'real', '1D'], ['canopystate_vars', 'altmax_lastyear_col', 'real', '1D'], ['decomp_cascade_con', 'spinup_factor', 'real', '1D'], ['decomp_cascade_con', 'is_cwd', 'logical', '1D'], ['cnstate_vars', 'scalaravg_col', 'real', '2D'], ['col_cf', 'decomp_cpools_sourcesink', 'real', '3D'], ['col_nf', 'decomp_npools_sourcesink', 'real', '3D'], ['col_pf', 'decomp_ppools_sourcesink', 'real', '3D'], ['cnstate_vars', 'som_diffus_coef_col', 'real', '2D'], ['cnstate_vars', 'som_adv_coef_col', 'real', '2D'], ['col_cs', 'decomp_cpools_vr', 'real', '3D'], ['col_ns', 'decomp_npools_vr', 'real', '3D'], ['col_ps', 'decomp_ppools_vr', 'real', '3D'], ['col_cf', 'decomp_cpools_transport_tendency', 'real', '3D'], ['col_nf', 'decomp_npools_transport_tendency', 'real',

In [12]:
for sub in subroutines.values():
    tree = sub.calltree[2:]
    sub.analyze_calltree(tree,case_dir)

soillittverttransp
|---->tridiagonal_sr
laketemperature
|---->soilthermprop_lake
|---->tridiagonal_sr
|---->phasechange_lake
soiltemperature
|---->soilthermprop
|---->computegroundheatfluxandderiv
|---->computeheatdifffluxandfactor
|---->solvetemperature
|---->|---->setrhsvec
|---->|---->|---->setrhsvec_snow
|---->|---->|---->|---->setrhsvec_snowurban
|---->|---->|---->|---->|---->setrhsvec_snowurbannonroad
|---->|---->|---->|---->|---->setrhsvec_snowurbanroad
|---->|---->|---->|---->setrhsvec_snownonurban
|---->|---->|---->setrhsvec_standingsurfacewater
|---->|---->|---->setrhsvec_soil
|---->|---->|---->|---->setrhsvec_soilurban
|---->|---->|---->|---->|---->setrhsvec_soilurbannonroad
|---->|---->|---->|---->|---->setrhsvec_soilurbanroad
|---->|---->|---->|---->setrhsvec_soilnonurban
|---->|---->|---->|---->setrhsvec_soil_standingsurfacewater
|---->|---->setmatrix
|---->|---->|---->setmatrix_snow
|---->|---->|---->|---->setmatrix_snowurban
|---->|---->|---->|---->|---->setmatrix_snowu